In [ ]:
import pandas as pd
df = pd.read_csv('library_1min.csv')
grouped = df.groupby(['SADDR', 'DADDR', 'PROTOCOL'])

network_flows = grouped.agg({'I-COUNT': 'sum', 'O-COUNT': 'sum'})

network_flows['SBYTES'] = grouped.apply(lambda x: x[x['TYPE'] == 'IN']['BYTES'].sum())
network_flows['SBYTES_MEAN'] = grouped.apply(lambda x: x[x['TYPE'] == 'IN']['BYTES'].mean())
network_flows['DBYTES'] = grouped.apply(lambda x: x[x['TYPE'] == 'OUT']['BYTES'].sum())
network_flows['DBYTES_MEAN'] = grouped.apply(lambda x: x[x['TYPE'] == 'OUT']['BYTES'].mean())


df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])

network_flows['TIME_DIFF_NS'] = grouped.apply(lambda x: (x['TIMESTAMP'].max() - x['TIMESTAMP'].min()).total_seconds() * 1e9)

network_flows.reset_index(inplace=True)

print(network_flows)

